### load derived data (jsonl file) into dictionary
This notebook shows how to:
- Pull data from the derived data file (JSON lines file)
- Extract the information we want, organize it into a dictionary
- Save the dictionary into a JSON file

After this has been stored into a JSON file, we can load it into pandas, cross check with unpaywall, etc. (all to be done in other notebooks)

In [1]:

import doi as doiLib
from unpywall.utils import UnpywallCredentials
from unpywall import Unpywall
from prepare_wos_data_to_json import Prepare as prp

In [2]:
dataPath = "../data/derived3/"
dataFile = "records.jsonl"

In [8]:
data = prp(dataPath+dataFile, dataPath)
data.run()
data.save("wos_data_as.json")

#import json
#with open(dataPath+"wos_data_as.json", 'w') as fp:
#    json.dump(data.dataDict, fp)